## __Versi 3a__
---

_(Basis implementasi bersumber dari versi 2)_

Ketentuan khusus versi 3A :
- Fokus pada Projection Head yang menerima hasil Transformer dengan shape (B, 99, 256)
- Melakukan GAP pada dim=1 di Projection Head (Cell 5)
- Melakukan proyeksi linear layer 256-128 di Projection Head (Cell 5)

Beberapa perubahan yang ditambahkan (ketentuan bersama) :

(PRETRAINING)
- Linear layer pada projection head diubah dari 256-512-128 menjadi 256-128
- Pada transformer, num_layers = 2 ditingkatkan menjadi num_layers = 5
- Menambahkan mekanisme early stopping dengan patience = 50 dan epoch = 200

(FINE-TUNING)
- Melakukan freeze pada 3D conv encoder, transformer, dan projection head
- Melatih classifier head saja

(EVALUASI)
- Hasil akhir berupa classification map

---

Cell yang terdampak modifikasi : 
- Cell 4 : Transformer
- Cell 5 : Projection Head
- Cell 11 : Inisialisasi
- Cell 12 : Training Loop
---

In [1]:
# Cell 1
# Import semua library dasar yang dibutuhkan

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt


In [2]:
# Cell 2
# Encoder 3D: bertugas mengekstraksi fitur spasial dan spektral dari patch hyperspectral.

# Tujuan: menghasilkan 99 token (3×3×11) berukuran embedding 256
# dari satu patch input berukuran (9×9×224)

class SpectralSpatialEncoder3D(nn.Module):
    def __init__(self, embedding_dim=256, init_channels=32):
        super().__init__()
        # -------------------------------------------------------
        # Konvolusi pertama:
        # - Kernel 3x3x20, stride sama (3x3x20)
        # - Non-overlapping, agar setiap kernel mencakup satu subpatch
        #   spasial-spektral unik
        # Output: (B, 32, 11, 3, 3)
        # -------------------------------------------------------
        self.conv1 = nn.Conv3d(
            in_channels=1,
            out_channels=init_channels,
            kernel_size=(20,3,3),
            stride=(20,3,3),
            padding=0
        )
        self.bn1 = nn.BatchNorm3d(init_channels)
        self.relu1 = nn.ReLU(inplace=True)

        # -------------------------------------------------------
        # Konvolusi kedua:
        # - Kernel 1x1x1, tanpa mengubah spasial/spektral
        # - Berfungsi sebagai "linear projection" ke embedding dim 256
        # Output: (B, 256, 11, 3, 3)
        # -------------------------------------------------------
        self.conv2 = nn.Conv3d(
            in_channels=init_channels,
            out_channels=embedding_dim,
            kernel_size=(1,1,1),
            stride=(1,1,1),
            padding=0
        )
        self.bn2 = nn.BatchNorm3d(embedding_dim)
        self.relu2 = nn.ReLU(inplace=True)
    
    def forward(self, x):
        # x: tensor input (B, 1, 224, 9, 9)
        x = self.relu1(self.bn1(self.conv1(x)))   # (B, 32, 11, 3, 3)
        x = self.relu2(self.bn2(self.conv2(x)))   # (B, 256, 11, 3, 3)

        # -------------------------------------------------------
        # Ubah urutan dimensi agar token berada dalam satu dimensi
        # Dari (B, 256, 11, 3, 3) ke (B, 11, 3, 3, 256)
        # Lalu diratakan menjadi (B, 99, 256)
        # -------------------------------------------------------
        B, C, D, H, W = x.shape

        # print("Encoder output before reshape:", x.shape)  # ← untuk debug
        x = x.permute(0,2,3,4,1).contiguous().view(x.size(0), -1, x.size(1)) # (B, 11, 3, 3, 256) lalu flatten ke (B, 99, 256)
        # print("Encoder output after reshape:", x.shape)   # ← untuk debug
        return x


In [3]:
# Cell 3
# Fungsi ini menambahkan noise Gaussian ke vektor embedding hasil encoder (featoken)
# untuk menghasilkan pasangan positif (positive key)

class LatentAugmentor:
    def __init__(self, sigma=0.1, device='cpu'):
        self.sigma = sigma
        self.device = device

    def __call__(self, featoken):
        """
        featoken: tensor berukuran (B, T, D)
        menghasilkan augmented_featoken: (B, T, D)
        """
        noise = torch.randn_like(featoken, device=featoken.device) * self.sigma
        return featoken + noise


In [4]:
# Cell 4
# Transformer Encoder sederhana.
# Menerima input dalam bentuk (batch, jumlah_token, dimensi_embedding) atau (B, T, D)
# Dalam desain sekarang, jumlah_token = T (karena 1 patch bisa menghasilkan token sebanyak T, yaitu 99)

class SimpleTransformerEncoder(nn.Module):
    def __init__(self, embed_dim=256, num_heads=8, num_layers=5, mlp_dim=512, dropout=0.1): # num_layer = 2 sudah diganti jadi 5
        super().__init__()
        layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            dim_feedforward=mlp_dim,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(layer, num_layers=num_layers)

    def forward(self, x):
        """
        x: tensor (B, T, D)
        T = jumlah token
        """
        return self.transformer(x)


In [5]:
# Cell 5
# Projection head memetakan output dari transformer ke dimensi ruang latent
# ruang latent adalah tempat dilakukan perhitungan kesamaan (cosine similarity).

# Ketentuan versi A ini : 
# - Proyeksi dilakukan dari 256 langsung ke 128
# - Pooling pada dim=1 dulu, lalu proyeksi 256 ke 128

class ProjectionHead_A(nn.Module): # ditambah tulisan "_A"
    def __init__(self, in_dim=256, proj_dim=128):
        super().__init__()
        self.net = nn.Linear(in_dim, proj_dim)

    def forward(self, x):
        """
        x shape: (B, 99, 256) dari transformer
        GAP antar token sehingga diperoleh hasil (B, 256)
        """
        x = x.mean(dim=1)  # Global Average Pool tiap dimensi embedding (secara horizontal - intertoken)
        return self.net(x)  # -> (B, 128)


In [6]:
# Cell 6
# Implementasi fungsi loss InfoNCE
# Mengukur kemiripan antar pasangan (query, positive key) dalam satu batch

def info_nce_loss(q, k, temperature=0.1):
    """
    q: queries (B, D)
    k: positive keys (B, D)
    """
    # Normalisasi
    q = F.normalize(q, dim=1)
    k = F.normalize(k, dim=1)

    # Hitung kesamaan antar semua pasangan dalam batch
    logits = torch.matmul(q, k.t()) / temperature
    labels = torch.arange(logits.size(0), device=logits.device)
    loss = F.cross_entropy(logits, labels)
    return loss


In [7]:
# Cell 7
# Load dataset hasil preprocessing

import os

data_dir = "../data/processed"
patch_class0_path = os.path.join(data_dir, "patch_class0.npy")
patch_class1_path = os.path.join(data_dir, "patch_class1.npy")

# Pastikan file ada
assert os.path.exists(patch_class0_path), f"File tidak ditemukan: {patch_class0_path}"
assert os.path.exists(patch_class1_path), f"File tidak ditemukan: {patch_class1_path}"

# Load file .npy
patch_class0 = np.load(patch_class0_path) # kelas non oil spill
patch_class1 = np.load(patch_class1_path) # kelas oil spill

# Cek ukuran masing-masing dataset
print("Class 0 shape:", patch_class0.shape)
print("Class 1 shape:", patch_class1.shape)

Class 0 shape: (5, 9, 9, 224)
Class 1 shape: (5, 9, 9, 224)


In [8]:
# Cell 8
# Gabungkan semua patch menjadi satu array
X_all = np.concatenate([patch_class0, patch_class1], axis=0)

# Buat label
y_all = np.concatenate([
    np.zeros(len(patch_class0)),  # label 0 untuk class 0
    np.ones(len(patch_class1))    # label 1 untuk class 1
])

print("Total samples:", X_all.shape[0])
print("Labels shape:", y_all.shape)


Total samples: 10
Labels shape: (10,)


In [9]:
# Cell 9
# Ubah dari numpy ke tensor
X_tensor = torch.tensor(X_all, dtype=torch.float32)
y_tensor = torch.tensor(y_all, dtype=torch.long)

# Ubah bentuk ke format Conv3D (N, C, D, H, W)
X_tensor = X_tensor.unsqueeze(1).permute(0, 1, 4, 2, 3)

print("Tensor shape setelah permute:", X_tensor.shape)


Tensor shape setelah permute: torch.Size([10, 1, 224, 9, 9])


In [10]:
# Cell 10
# Split train dan validation

# Tentukan ukuran train dan validation
train_size = int(0.8 * len(X_tensor))
val_size = len(X_tensor) - train_size

# Buat indeks acak untuk memastikan X dan y sejajar
indices = torch.randperm(len(X_tensor))
train_idx = indices[:train_size]
val_idx = indices[train_size:]

# Bagi data berdasarkan indeks yang sama
train_X = X_tensor[train_idx]
train_y = y_tensor[train_idx]
val_X = X_tensor[val_idx]
val_y = y_tensor[val_idx]

# Buat TensorDataset
train_dataset = TensorDataset(train_X, train_y)
val_dataset = TensorDataset(val_X, val_y)

# Buat DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

print(f"Train samples: {len(train_dataset)} | Validation samples: {len(val_dataset)}")


Train samples: 8 | Validation samples: 2


In [11]:
# Cell 11
# Inisialisasi perangkat dan model
# Inisialisasi perangkat
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Inisialisasi model
encoder = SpectralSpatialEncoder3D(embedding_dim=256).to(device)
augmentor = LatentAugmentor(sigma=0.08) # tidak .to(device) karena tidak trainable
transformer = SimpleTransformerEncoder(embed_dim=256).to(device)

proj_head = ProjectionHead_A(in_dim=256, proj_dim=128).to(device)  # menggunakan ProjectionHead_A
   

# Parameter dan optimizer
params = list(encoder.parameters()) + list(transformer.parameters()) + list(proj_head.parameters())
optimizer = optim.AdamW(params, lr=1e-4, weight_decay=0.01)



In [12]:
# Cell 12
# Training loop dengan validasi per epoch (dengan checkpoint & resume)

import os
import time
from tqdm import tqdm

# ==== PARAMETER ====
START_EPOCH = 1          # default, akan ditimpa otomatis jika ada checkpoint
NUM_EPOCHS = 200         # epoch di-set 200

patience = 50           # Keperluan early stopping
no_improve = 0          # Keperluan early stopping

temperature = 0.1
best_val_loss = float('inf')
checkpoint_path = "checkpoint_sst_ver3A.pt" # nama file checkpoint diperjelas ver3A

# ==== Jika checkpoint ada, lanjutkan dari sana ====
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device)
    encoder.load_state_dict(checkpoint["encoder_state"])
    transformer.load_state_dict(checkpoint["transformer_state"])
    proj_head.load_state_dict(checkpoint["proj_head_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])
    START_EPOCH = checkpoint["epoch"] + 1
    best_val_loss = checkpoint["best_val_loss"]
    print(f"[OK] Checkpoint ditemukan. Melanjutkan dari epoch {START_EPOCH}.")
else:
    print("[MAAF] Tidak ditemukan checkpoint. Memulai training dari awal.")

# ==== Mulai Training ====
for epoch in range(START_EPOCH, NUM_EPOCHS+1):  # Start awal di 1, hingga nanti di 201 - 1
    start_time = time.time()

    # ------------------------
    # MODE TRAIN
    # ------------------------
    encoder.train()
    transformer.train()
    proj_head.train()

    total_train_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{NUM_EPOCHS} [Train]", leave=True)
    for batch_X, _ in pbar:
        batch_X = batch_X.to(device)

        # 1. Encode patch untuk dapatkan token embedding (B, 99, 256)
        featoken = encoder(batch_X)

        # 2. Latent augmentation (Gaussian Noise)
        aug_featoken = augmentor(featoken)

        # 3️. Proses lewat transformer (attention antar token), output transformer shape (B, 99, 256) / (B, T, D)
        z_orig = transformer(featoken)
        z_aug  = transformer(aug_featoken)


        # 4️. Proyeksikan ke ruang latent (untuk InfoNCE), output proj_head shape (B, 128) / (B, proj_dim)
        q = proj_head(z_orig)
        k = proj_head(z_aug)

        # 5. Hitung loss contrastive
        loss = info_nce_loss(q, k, temperature=temperature)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_train_loss += loss.item()
        pbar.set_postfix({"Train Loss": f"{loss.item():.4f}"})

    avg_train_loss = total_train_loss / len(train_loader)

    # ------------------------
    # MODE VALIDASI
    # ------------------------
    encoder.eval()
    transformer.eval()
    proj_head.eval()
    total_val_loss = 0.0

    with torch.no_grad():
        for batch_X, _ in val_loader:
            batch_X = batch_X.to(device)
            featoken = encoder(batch_X)
            aug_featoken = augmentor(featoken)
            z_orig = transformer(featoken)
            z_aug  = transformer(aug_featoken)
            q = proj_head(z_orig)
            k = proj_head(z_aug)
            val_loss = info_nce_loss(q, k, temperature)
            total_val_loss += val_loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    
    # Waktu per epoch
    epoch_time = time.time() - start_time

    print(f"Epoch [{epoch}/{NUM_EPOCHS}] "
          f"Train Loss: {avg_train_loss:.4f} | "
          f"Val Loss: {avg_val_loss:.4f} | "
          f"Time: {epoch_time:.2f}s")

    # ------------------------
    # SIMPAN CHECKPOINT (checkpoint memiliki file-nya sendiri)
    # ------------------------
    checkpoint = {
        "epoch": epoch,
        "encoder_state": encoder.state_dict(),
        "transformer_state": transformer.state_dict(),
        "proj_head_state": proj_head.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "best_val_loss": best_val_loss
    }
    torch.save(checkpoint, checkpoint_path)

    # Simpan model terbaik (model terbaik memiliki file-nya sendiri)
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        no_improve = 0
        torch.save(checkpoint, "best_sst_ver3A.pt")  # nama file best model diperjelas ver3A
        print("OK Model terbaik disimpan.")
    else:
        no_improve += 1
        print(f"[Epoch {epoch}] No improvement {no_improve}/{patience}.")

    if no_improve >= patience:
        print("Early stopping triggered.")
        break

print("Alhamdulillah, Training selesai.")



[MAAF] Tidak ditemukan checkpoint. Memulai training dari awal.


Epoch 1/200 [Train]: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00,  1.83it/s, Train Loss=1.0726]


Epoch [1/200] Train Loss: 1.0726 | Val Loss: 0.1779 | Time: 0.58s
OK Model terbaik disimpan.


Epoch 2/200 [Train]: 100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 11.29it/s, Train Loss=0.7160]


Epoch [2/200] Train Loss: 0.7160 | Val Loss: 0.0834 | Time: 0.09s
OK Model terbaik disimpan.


...

Epoch [91/200] Train Loss: 0.0006 | Val Loss: 0.0000 | Time: 0.04s


[Epoch 91] No improvement 49/50.


Epoch 92/200 [Train]: 100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 29.64it/s, Train Loss=0.0006]

Epoch [92/200] Train Loss: 0.0006 | Val Loss: 0.0000 | Time: 0.04s


[Epoch 92] No improvement 50/50.
Early stopping triggered.
Alhamdulillah, Training selesai.
